In [10]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random
import pickle
import time
import xlsxwriter

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set api token 
QX_TOKEN = "96c86855a5c381875a4ea3a956a7deafbbbbb82e5912ec73655c7c444d2597a7364eda2fa558e4e37f672640029443216899825f945d5a0684e31b796837ce61"
QX_URL = "https://quantumexperience.ng.bluemix.net/api"


#register token and url
register(QX_TOKEN, QX_URL)

backend = 'local_qasm_simulator'

# prepare a state of all zeros
def prepareZeros(qc):
    qc.h(qr[1])
    qc.barrier()
    qc.cx(qr[1],qr[0])
    qc.barrier()
    qc.cx(qr[1],qr[2])
    qc.barrier()
    qc.cx(qr[2],qr[3])
    qc.barrier()

    
# compile into different states
def compilation(qc,i,perm):
    if i == 0: 
        return
    elif i == 1:
        qc.x(qr[0])
        qc.x(qr[2])
    elif i == 2:
        qc.x(qr[0])
        qc.x(qr[1])
    elif i == 3:
        qc.x(qr[2])
        qc.x(qr[1])
    qc.barrier()
            
# add phase gate
def addPhase(qc):
    qc.h(qr[0])
    qc.h(qr[1])
    qc.h(qr[2])
    qc.h(qr[3])
    qc.barrier()
    qc.cx(qr[1],qr[0])
    qc.barrier()
    qc.s(qr[0])
    qc.barrier()
    qc.cx(qr[1],qr[0])
    qc.barrier()
    
# remove phase gate
def removePhase(qc,perm):
    qc.cx(qr[perm[1]], qr[perm[0]])
    qc.barrier()
    qc.sdg(qr[perm[0]])
    qc.barrier()
    qc.cx(qr[perm[1]], qr[perm[0]])
    qc.barrier()
    qc.h(qr[3])
    qc.h(qr[2])
    qc.h(qr[1])
    qc.h(qr[0])
    qc.barrier()
    
# parse a string of gens
def addCliffGens(qc, string, perm):
    for char in string:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[2]])
        elif i==1:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
        elif i==3:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[2]])
        elif i ==4:
            qc.h(qr[0])
            qc.h(qr[1])
            qc.h(qr[2])
            qc.h(qr[3])
        elif i == 5:
            qc.s(qr[0])
            qc.s(qr[1])
            qc.s(qr[2])
            qc.s(qr[3])
        elif i==6:
            swap(perm,0,1)
        elif i==7:
            swap(perm,0,2)        

# find string for generators
def findGenString(mat):
    for key, value in signedGroup.items():
        if np.array_equal(value, mat):
            return key
    print("OH NO")
    return None
        
# invert cliff gens 
def invertCliffGens(qc, mat, perm):
    genString = findGenString(mat)
    for char in genString[::-1]:
        i = int(char)
        if i==0:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[2]])
        elif i==1:
            qc.x(qr[perm[0]])
            qc.x(qr[perm[1]])
        elif i==2:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[1]])
        elif i==3:
            qc.z(qr[perm[0]])
            qc.z(qr[perm[2]])
        elif i ==4:
            qc.h(qr[0])
            qc.h(qr[1])
            qc.h(qr[2])
            qc.h(qr[3])
        elif i == 5:
            qc.sdg(qr[0])
            qc.sdg(qr[1])
            qc.sdg(qr[2])
            qc.sdg(qr[3])
        elif i==6:
            swap(perm,0,1)
        elif i==7:
            swap(perm,0,2)
    qc.barrier()

# adds random real clifford
def addRandCliff(qc, mat,perm):
    genString = random.choice(list(r2group.keys()))
    addCliffGens(qc,genString,perm)
    qc.barrier()
    return np.matmul(mat,r2group[genString])

pickle_in = open("r2group.pkl", "rb")
r2group = pickle.load(pickle_in)
pickle_in2 = open("r2group.pickle", "rb")
signedGroup = pickle.load(pickle_in2)

perm = {0: 0, 1: 1, 2: 2, 3: 3}

# used for reindexing qubits
def swap(perm, a, b):
    temp = perm[a]
    perm[a] = perm[b]
    perm[b] = temp
    
# converts 0-3 to binary strings
def toBinString(num):
    if num ==0:
        return "00"
    elif num == 1:
        return "01"
    elif num == 2:
        return "10"
    elif num == 3:
        return "11"
    else:
        print("OH NO")

def printDataFormatted(data):
    strings = ["0000", "1111", "0011", "1100", "1010", "0101", "1001", "0110", "1000", "0111", "0001", "1110"
              , "0010", "1101", "0100", "1011"]
    for string in strings:
        try:
            print(string+": "+str(data[string]))
        except:
            print("oops, none of "+string)
            pass
    

def writeToExcel(results,row,col):
    states = ["0000", "1111", "0011", "1100", "1010", "0101", "1001", "0110", "1000", "0111", "0001", "1110"
              , "0010", "1101", "0100", "1011"]
    for state in states:
        try:
            print(results[state])
            worksheet.write(row,col,results[state])
        except:
            print(state+"does not exist")
            worksheet.write(row,col,0)
        col = col+1
    print(row)
    print(col)
    col = 1
    return row+1
        
        
lengths = [3]
reps = [3,9,9,9]
initialStates = [0]
shots = 1024
phased = True
shouldWrite = True

if shouldWrite:
    workbook = xlsxwriter.Workbook("coded_("+
                                   ",".join(map(str,lengths))+
                                   ")_("+
                                   ",".join(map(str,reps))+
                                   ")_ "+
                                   str(int(phased))+".xlsx")
    worksheet = workbook.add_worksheet()
    
    states = ["0000", "1111", "0011", "1100", "1010", "0101", "1001", "0110", "1000", "0111", "0001", "1110"
              , "0010", "1101", "0100", "1011"]
    for index in range(len(states)):
        worksheet.write(0,index+1,states[index])
    row = 1
    col = 1

startTime = time.time()

for compiler in initialStates:

    for length in lengths:
        
        print(str(length)+" "+str(compiler)+"\n")

        # run our trials
        for i in range(reps[compiler]):

            # run one trial of given length 
            mat = r2group[""]
            n = 4
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr, cr)
            
            # prepare encoded zeros
            prepareZeros(qc) 
            
            # optional: add phase
            if phased:
                addPhase(qc)

            # add the first m-1 clifford gates
            for j in range(length-1):
                mat = addRandCliff(qc,mat,perm)

            # invert everything
            invertCliffGens(qc,mat, perm)
            
            # optional: remove phase
            if phased:
                removePhase(qc,perm)

            # get the qc compiled into one of the 4 different states
            compilation(qc,compiler, perm)
            

            # measure
            for i in range(n):
                qc.measure(qr[perm[i]], cr[i])      
            
            print("got")
            job = execute(qc, backend, shots = shots)
            print("here")
            result = job.result()
            print("nice")
            trial = result.get_counts(qc)
            
            # this is now outdated
            #printDataFormatted(trial)
            
            # clean the data
            data = {}
            totalSuccessful = 0
            for key in trial:
                ohoh = ['0000', '1111']
                ohone = ['1100', '0011']
                oneoh = ['1010', '0101']
                oneone = ['0110', '1001']
                if key not in ohoh+ohone+oneoh+oneone:
                    continue
                if key in ohoh:
                    if "00" not in data.keys():
                        data["00"] = 0
                    data["00"] += trial[key]
                    totalSuccessful += trial[key]
                elif key in ohone:
                    if "01" not in data.keys():
                        data["01"] = 0
                    data["01"] += trial[key]
                    totalSuccessful += trial[key]
                elif key in oneoh:
                    if "10" not in data.keys():
                        data["10"] = 0
                    data["10"] += trial[key]
                    totalSuccessful += trial[key]
                elif key in oneone:
                    if "11" not in data.keys():
                        data["11"] = 0
                    data["11"] += trial[key]
                    totalSuccessful += trial[key]
            #plot_histogram(data)
            
            # set shouldWrite to false if running on simulator or don't actually care
            if shouldWrite:
                row = writeToExcel(trial,row,col)
            
            # optional: get success rate
            desired = toBinString(compiler)
            print(data[desired]/totalSuccessful)
            

endTime = time.time()
print("Time elapsed: ", endTime-startTime)

if shouldWrite:
    workbook.close()


  

Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\quantum_rb\Qconfig.py.
3 0

got
here
nice
0000: 521
1111: 503
oops, none of 0011
oops, none of 1100
oops, none of 1010
oops, none of 0101
oops, none of 1001
oops, none of 0110
oops, none of 1000
oops, none of 0111
oops, none of 0001
oops, none of 1110
oops, none of 0010
oops, none of 1101
oops, none of 0100
oops, none of 1011
521
503
0011does not exist
1100does not exist
1010does not exist
0101does not exist
1001does not exist
0110does not exist
1000does not exist
0111does not exist
0001does not exist
1110does not exist
0010does not exist
1101does not exist
0100does not exist
1011does not exist
1
17
1.0
got
here
nice
0000: 521
1111: 503
oops, none of 0011
oops, none of 1100
oops, none of 1010
oops, none of 0101
oops, none of 1001
oops, none of 0110
oops, none of 1000
oops, none of 0111
oops, none of 0001
oops, none of 1110
oops, none of 0010
oops, none of 1101
oops, none of 0100
oops, none of 1011
521
503
0011does not exist
1100does n